<a href="https://colab.research.google.com/github/FaddliLWibowo/-Twitter-Sentiment-Analysis-with-Streamlit/blob/main/Data_Collection_From_Twitter_For_Sentiment_Analysis_With_Emoji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snscrape==0.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 KB 456.2 kB/s eta 0:00:00


In [2]:
import snscrape.modules.twitter as sntwitter
import datetime as dt
import pandas as pd

In [49]:
def scrape_tweet(search_term, start_date, end_date, num_tweets):
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")
    tweet_data = []
    for i, tweet in enumerate(
        sntwitter.TwitterSearchScraper(
            "{} since:{} until:{} lang:id exclude:retweets".format(
                search_term, start_date, end_date
            )
        ).get_items()
    ):
        if i >= num_tweets:
            break
        tweet_data.append([tweet.date, tweet.user.username, tweet.content])
    tweet_df = pd.DataFrame(tweet_data, columns=["tweetcreatedts", "username", "text"])
    return tweet_df

In [25]:
def daily_scraping(search_term, num_daily):
    start_date = dt.datetime(2023, 2, 1)
    end_date = dt.datetime(2023, 2, 28)
    delta = dt.timedelta(days=1)
    df = pd.DataFrame()
    for n in range(7):
        temp_df = scrape_tweet(search_term, start_date, end_date, num_daily)
        df = pd.concat([df, temp_df])
        start_date += delta
        end_date += delta
    return df

In [26]:
ori_neg_df = daily_scraping(":(", 500)

<ipython-input-24-d6b75cfba204>:14: FutureWarning: content is deprecated, use rawContent instead
  tweet_data.append([tweet.date, tweet.user.username, tweet.content, tweet.source])


In [27]:
ori_pos_df = daily_scraping(":)", 500)

<ipython-input-24-d6b75cfba204>:14: FutureWarning: content is deprecated, use rawContent instead
  tweet_data.append([tweet.date, tweet.user.username, tweet.content, tweet.source])


In [34]:
def filter_include(df, term_list):
    temp_df = pd.DataFrame()
    for term in term_list:
        add_df = df[df["text"].str.contains(term, regex=False) == True]
        temp_df = pd.concat([temp_df, add_df]).drop_duplicates(ignore_index=True)
    return temp_df

In [35]:
def filter_exclude(df, term_list):
    temp_df = df.copy()
    for term in term_list:
        temp_df = temp_df[temp_df["text"].str.contains(term, regex=False) == False]
    return temp_df

In [36]:
neg_df = filter_include(ori_neg_df, [":(", ":-("])
neg_df = filter_exclude(neg_df, [":)", ":D", ":-)"])
neg_df.shape

(499, 4)

In [37]:
pos_df = filter_include(ori_pos_df, [":)", ":D", ":-)"])
pos_df = filter_exclude(pos_df, [":(", ":-("])
pos_df.shape

(487, 4)

In [40]:
def remove_term(df, term_list):
    temp_df = df.copy()
    for term in term_list:
        temp_df["text"] = temp_df["text"].str.replace(term, " ", regex=False)
    return temp_df

In [41]:
neg_df = remove_term(neg_df, [":(", ":-("])

In [42]:
pos_df = remove_term(pos_df, [":)", ":D", ":-)"])

In [43]:
neg_df["klasifikasi"] = "Negatif"
pos_df["klasifikasi"] = "Positif"
df = pd.concat([neg_df, pos_df]).reset_index(drop=True)

In [46]:
#Mengganti format tanggal
tanggal = pd.to_datetime(df['tweetcreatedts'])
df['tweetcreatedts'] = tanggal.dt.strftime('%d/%m/%Y %H:%M')
df['tweetcreatedts']

0      27/02/2023 22:29
1      27/02/2023 22:29
2      27/02/2023 22:29
3      27/02/2023 22:28
4      27/02/2023 22:28
             ...       
981    27/02/2023 21:57
982    27/02/2023 21:38
983    27/02/2023 21:35
984    27/02/2023 21:33
985    27/02/2023 21:30
Name: tweetcreatedts, Length: 986, dtype: object

In [47]:
df

,tweetcreatedts,username,text,source,klasifikasi
0,27/02/2023 22:29,gemini4thism,"Mau nangis, kangen Fourth","<a href=""http://twitter.com/download/iphone"" r...",Negatif
1,27/02/2023 22:29,nico_inoo,dingin bgt ky gini jadi tiba² keinget pas kmu ...,"<a href=""http://twitter.com/download/android"" ...",Negatif
2,27/02/2023 22:29,txtfromholy,@AgneesLS Banget,"<a href=""http://twitter.com/download/iphone"" r...",Negatif
3,27/02/2023 22:28,notanicecup,Kenapa bangun tidur bisa kerasa bgt kangennya,"<a href=""http://twitter.com/download/android"" ...",Negatif
4,27/02/2023 22:28,kzm_rnk,"@rrwynna Iyaaa, kecuali kalo kayak event yang ...","<a href=""http://twitter.com/download/android"" ...",Negatif
...,...,...,...,...,...
981,27/02/2023 21:57,test5f1798,foo : 180501cc-9760-4654-b18b-92566cf03d9a,"<a href=""http://twitter.com"" rel=""nofollow"">Te...",Positif
982,27/02/2023 21:38,ViaKyuTemErtDae,I'am Vhia ELF from INA always support SUPER JU...,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",Positif
983,27/02/2023 21:35,DORKyungsoo4,@zioshaiendaryll Jadi begitu,"<a href=""http://twitter.com/download/iphone"" r...",Positif
984,27/02/2023 21:33,Rizqiazulvaafia,Heyy semangat yaa coboyjr iqbaale teukuryzki99...,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",Positif


In [48]:
df.to_excel("labeled_tweets.xlsx", index=False)